In [42]:
import numpy as np
import pandas as pd
from my_stuff import MufexKeys
from quantfreedom.enums import *
from quantfreedom.exchanges.mufex_exchange.mufex import Mufex
from quantfreedom.indicators.tv_indicators import *
from quantfreedom.plotting.plot_funcs import *
from quantfreedom.exchanges.exchange import Exchange
from quantfreedom.helper_funcs import dl_ex_candles, candles_to_df
np.set_printoptions(formatter={"float_kind": "{:0.1f}".format})

mufex_main = Mufex(use_test_net=False)

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [43]:
candles = mufex_main.get_candles(symbol="BTCUSDT", timeframe="5m", candles_to_dl=400)
open = candles[:1]
high = candles[:, 2]
low = candles[:, 3]
close = candles[:, 4]
close_shift = np.roll(candles[:, 4],1)

It took 00 mins and 01 seconds to download 400 candles


In [44]:
dl_ex_candles(
    exchange="mufex",
    symbol="BTCUSDT",
    timeframe="5m",
    candles_to_dl=400,
)

It took 00 mins and 00 seconds to download 400 candles


array([[1699360800000.0, 34706.8, 34732.1, 34706.8, 34721.0],
       [1699361100000.0, 34721.0, 34738.7, 34712.8, 34738.3],
       [1699361400000.0, 34738.3, 34755.3, 34720.7, 34740.1],
       ...,
       [1699479900000.0, 35486.6, 35556.8, 35476.6, 35554.8],
       [1699480200000.0, 35554.8, 35555.1, 35522.0, 35550.4],
       [1699480500000.0, 35550.4, 35599.9, 35538.4, 35592.2]])

In [46]:
candles_to_df(candles=candles)

,timestamp,open,high,low,close
datetime,,,,,
2023-11-07 12:40:00,1699360800000,34706.8,34732.1,34706.8,34721.0
2023-11-07 12:45:00,1699361100000,34721.0,34738.7,34712.8,34738.3
2023-11-07 12:50:00,1699361400000,34738.3,34755.3,34720.7,34740.1
2023-11-07 12:55:00,1699361700000,34740.1,34747.7,34724.4,34741.8
2023-11-07 13:00:00,1699362000000,34741.8,34765.3,34741.8,34750.2
...,...,...,...,...,...
2023-11-08 21:35:00,1699479300000,35513.9,35520.3,35424.5,35471.0
2023-11-08 21:40:00,1699479600000,35471.0,35494.7,35407.4,35486.6
2023-11-08 21:45:00,1699479900000,35486.6,35556.8,35476.6,35554.8
